Name : Ng Chen Ting 
# Part B
## Task 1 - Processing Data Stream
### Event Producer 1

In [ ]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
from pymongo import MongoClient
from datetime import datetime
import pandas
import random
import datetime as dt
import csv

hostip = "192.168.1.14" 
client = MongoClient(hostip, 27017) 
db = client["fit3182_assignment_db"] 
climate_collection = db["Assignment_2_Climates"]

def load_data_from_csv():
    """
    Load data from 'climate_streaming.csv' and format it into a list of dictionaries.
    """
    climate_streaming_data = pandas.read_csv('climate_streaming.csv') 
    data = []
    
    # Iterate through each row 
    for _, row in climate_streaming_data.iterrows(): 
        data_row = {} 
        data_row['latitude'] = float(row['latitude'])
        data_row['longitude'] = float(row['longitude'])
        data_row['air_temperature_celcius'] = int(row['air_temperature_celcius'])
        data_row['relative_humidity'] = float(row['relative_humidity'])
        data_row['windspeed_knots'] = float(row['windspeed_knots'])
        data_row['max_wind_speed'] = float(row['max_wind_speed'])
        data_row['precipitation'] = str(row['precipitation '])
        data_row['GHI_w/m2'] = int(row['GHI_w/m2'])
        data.append(data_row)
    return data

def publish_message(producer_instance, topic_name, value):
    try:
        # To convert them to bytes 
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, value=value_bytes)

        # To ensure the message is actually sent to Kafka 
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(value))
        
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
    
if __name__ == '__main__':
    topic = 'Assignment_TaskB_Climate'
    print('Publishing records..')
    producer = connect_kafka_producer()
    data = load_data_from_csv()
    
    # Initialise current date retrieved above
    latest_document = climate_collection.find_one(sort=[("date", -1)])  # Sort Date in Descending order
    current_date = latest_document['date']

    while True:
        try:
            # Get a random line from 'climate_streaming.csv'
            random_line = random.choice(data)

            # Increment by a day
            current_date += dt.timedelta(days=1)
            random_line['date'] = current_date.strftime("%d/%m/%Y") ### Update date
            
            # Add producer_id to indicate it's from climate producer
            random_line['producer_id'] = 'producer_climate' 

            # Convert the dictionary to a JSON string and publish
            json_data = dumps(random_line) 
            publish_message(producer, topic, json_data) 
            sleep(10) # Wait 10 seconds

        except KeyboardInterrupt:
            print('Exiting...')
            break


Publishing records..
Message published successfully. Data: {"latitude": -37.396, "longitude": 148.086, "air_temperature_celcius": 15, "relative_humidity": 41.6, "windspeed_knots": 15.0, "max_wind_speed": 28.0, "precipitation": " 0.00I", "GHI_w/m2": 138, "date": "08/06/2024", "producer_id": "producer_climate"}
Message published successfully. Data: {"latitude": -35.961, "longitude": 141.089, "air_temperature_celcius": 5, "relative_humidity": 34.9, "windspeed_knots": 4.3, "max_wind_speed": 11.1, "precipitation": " 0.00A", "GHI_w/m2": 48, "date": "09/06/2024", "producer_id": "producer_climate"}
Message published successfully. Data: {"latitude": -37.605, "longitude": 149.326, "air_temperature_celcius": 23, "relative_humidity": 49.6, "windspeed_knots": 10.0, "max_wind_speed": 19.0, "precipitation": " 0.00I", "GHI_w/m2": 198, "date": "10/06/2024", "producer_id": "producer_climate"}
Message published successfully. Data: {"latitude": -37.375, "longitude": 148.054, "air_temperature_celcius": 12,